In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [2]:
import pickle

def pickle_store(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
        
def pickle_load(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [3]:
movies_path = os.path.join("ml-25m", "movies.csv")
ratings_path = os.path.join("ml-25m", "ratings.csv")

user_mean_path = os.path.join("ml-25m", "user_mean.csv")
ratings_avg_path = os.path.join("ml-25m", "ratings_avg.csv")
ratings_avg2_path = os.path.join("ml-25m", "ratings_real_avg.csv")

final_path = os.path.join("ml-25m", "final.csv")
cosine_path = os.path.join("ml-25m", "cosine.csv")

pivot_table_path = os.path.join("ml-25m", "pivot_table.sparse")

In [4]:
iteration = 0
movies_ids = np.array([])
users_ids = np.array([])
headers_tm = time.time()
for chunk in pd.read_csv(ratings_path, chunksize=100000):
    movies_ids = np.append(movies_ids, chunk['movieId'].unique())
    users_ids = np.append(users_ids, chunk['userId'].unique())
    iteration += 1
movies_ids = np.unique(movies_ids) 
users_ids = np.unique(users_ids) 
print("Took {:.3f}".format(time.time()-headers_tm))

Took 8.565


In [34]:
print("iterations: ", iteration)
print("dif movies are: ", movies_ids.shape)
print("dif users are: ", users_ids.shape)

iterations:  2501
dif movies are:  (59047,)
dif users are:  (162541,)


In [6]:
starting_tm = time.time()
pivot_table = csr_matrix((len(users_ids), len(movies_ids)))
for chunk in pd.read_csv(ratings_path, chunksize=100000):
    loop_tm = time.time()
    rows = [i-1 for i in chunk['userId'].tolist()]
    cols = [np.where(movies_ids == j)[0][0] for j in chunk['movieId']]
    vals = chunk['rating'].tolist()
    pivot_table[rows, cols] = vals
    # print("Took {:.3f}".format(time.time()-loop_tm))
print("Took {:.3f}".format(time.time()-starting_tm))

/home/teomandi/anaconda3/envs/project1/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Took 918.479


In [5]:
#pickle_store(pivot_table, pivot_table_path)
pivot_table = pickle_load(pivot_table_path)
print("PivoTable was stored")

PivoTable was stored


In [14]:
similarity_tm = time.time()
# similarity_matrix = cosine_similarity(pivot_table) ## BRUTAL
print("Took {:.3f}".format(time.time()-similarity_tm))

Took 0.000


In [7]:
order = np.argsort(user_similarities, axis=0)
order[:5]

array([[143635],
       [154450],
       [154453],
       [ 14980],
       [ 14977]])

---

## Getting Predictions

In [12]:
Rating_avg = pd.read_csv(ratings_path)
Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))

### Get target and similar users

In [ ]:
target_user_id =  1

similarity_tm = time.time()
user_similarities = cosine_similarity(pivot_table, pivot_table.getrow(target_user_id-1))
print("Took {:.3f}".format(time.time()-similarity_tm))
print(user_similarities.shape)
user_similarities

In [40]:
similar_users = (-user_similarities).argsort(axis=0)
most_similar_users = (similar_users[1:11]).squeeze().tolist()  # the first is itself

### Get the movies which similar users has seen
    

In [52]:
movies_seen_by_similar_users = []
for user in most_similar_users:
    movies_seen_by_similar_users.extend(pivot_table.getrow(user).nonzero()[1])
movies_seen_by_similar_users = set(movies_seen_by_similar_users)
print(len(movies_seen_by_similar_users))

732


In [53]:
movies_seen_by_target_user = pivot_table.getrow(target_user_id-1).nonzero()[1]
print(len(movies_seen_by_target_user))

70


In [54]:
movies_under_consideration = list(set(movies_seen_by_similar_users) - set(movies_seen_by_target_user))
print(len(movies_under_consideration))

717


In [61]:
movie_avg_ratings = []
for movie in movies_under_consideration:
    movie_ratings = pivot_table[most_similar_users, movie].toarray().squeeze().tolist()
    movie_avg_ratings.append(sum(movie_ratings)/len(movie_ratings))

In [62]:
best_movies_indexes = np.array(movie_avg_ratings).argsort()[:4].tolist()
predictions = [int(movies_ids[idx]) for idx in best_movies_indexes]
print(predictions)


[138, 598, 170, 104]
